In [1]:
import torchtext
from torchtext.data.utils import get_tokenizer
import torch
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [21]:

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    print("before batchify:",data.size())
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    print("after batchify:",data.size())
    return data.to(device)

batch_size = 20
eval_batch_size = 10
train_data = batchify(train_txt, batch_size)
bptt = 35

def get_batch(source, i):
    print(source.size())
    print(source)
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

data,target = get_batch(train_data,0)

print(data.size())
print(target)

before batchify: torch.Size([2086708, 1])
after batchify: torch.Size([104335, 20])
torch.Size([104335, 20])
tensor([[    3,    25,  1849,  ...,     5,    65,    30],
        [   12,    66,    13,  ...,    35,  2438,  4064],
        [ 3852, 13667,  2962,  ...,   902,    33,    20],
        ...,
        [  154,     7,    10,  ...,     5,  1076,    78],
        [   25,     4,  4135,  ...,     4,    56,   299],
        [    6,    57,   385,  ...,  3168,   737,    36]], device='cuda:0')
torch.Size([35, 20])
torch.Size([700])
